In [11]:
%run -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 50.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
from MyCapytain.resources.texts.local.capitains.cts import CapitainsCtsText
import spacy

nlp = spacy.load("en_core_web_sm", disable=["ner"])

with open("tlg0012.tlg002.perseus-eng4.xml") as f:
    text_2 = CapitainsCtsText(urn="urn:cts:greekLit:tlg0012.tlg002.perseus-eng4", resource=f)

from lxml import etree
from MyCapytain.common.constants import Mimetypes

urns = []
raw_xmls = []
unannotated_strings = []

for ref in text_2.getReffs(level=len(text_2.citation)):
    urn = f"{text_2.urn}:{ref}"
    node = text_2.getTextualNode(ref)
    raw_xml = node.export(Mimetypes.XML.TEI)
    tree = node.export(Mimetypes.PYTHON.ETREE)
    s = etree.tostring(tree, encoding="unicode", method="text")

    urns.append(urn)
    raw_xmls.append(raw_xml)
    unannotated_strings.append(s)

import pandas as pd

d_2 = {
    "urn": pd.Series(urns, dtype="string"),
    "raw_xml": raw_xmls,
    "unannotated_strings": pd.Series(unannotated_strings, dtype="string")
}
homer_df = pd.DataFrame(d_2)

raw_texts = [t for t in homer_df['unannotated_strings']]

annotated_texts = nlp.pipe(raw_texts, batch_size=100)

# we want to count Son and son in our son count, lowering cases of all unannotated_strings does that
homer_df['unannotated_strings'] = homer_df['unannotated_strings'].str.lower()

homer_df['nlp_docs'] = list(annotated_texts)

n_son_homer = [t for t in homer_df['nlp_docs'].explode() if t.lemma_ == "son"]
#print(n_son_homer)
#homer_son_lines['son_count'] = [t for t in homer_son_lines['nlp_docs'].explode() if t.lemma_ == "son"]
# transform unannotated strings to all lowercase

homer_son_lines = homer_df.loc[homer_df["unannotated_strings"].str.contains('son', regex =False)]
homer_son_lines['unannotated_tokens'] = homer_son_lines["unannotated_strings"].str.replace('"', '', regex=False).str.split()
homer_son_lines['son_count'] = homer_son_lines['unannotated_tokens'].apply(lambda x: sum([1 for y in x if y == 'son']))

#str.count('son')
homer_son_lines


/tmp/ipykernel_2548/1165739777.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  homer_son_lines['unannotated_tokens'] = homer_son_lines["unannotated_strings"].str.replace('"', '', regex=False).str.split()
/tmp/ipykernel_2548/1165739777.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  homer_son_lines['son_count'] = homer_son_lines['unannotated_tokens'].apply(lambda x: sum([1 for y in x if y == 'son']))


,urn,raw_xml,unannotated_strings,nlp_docs,unannotated_tokens,son_count
0,urn:cts:greekLit:tlg0012.tlg002.perseus-eng4:1.1,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...","tell me, o muse, of that many-sided hero who ...","(Tell, me, ,, O, Muse, ,, of, that, many, -, s...","[tell, me,, o, muse,, of, that, many-sided, he...",1
1,urn:cts:greekLit:tlg0012.tlg002.perseus-eng4:1.44,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...","then athena said, ""father, son of kronos, king...","(Then, Athena, said, ,, "", Father, ,, son, of,...","[then, athena, said,, father,, son, of, kronos...",4
3,urn:cts:greekLit:tlg0012.tlg002.perseus-eng4:1...,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...","""i hope, sir,"" said he, ""that you will not be ...","("", I, hope, ,, sir, ,, "", said, he, ,, "", tha...","[i, hope,, sir,, said, he,, that, you, will, n...",4
4,urn:cts:greekLit:tlg0012.tlg002.perseus-eng4:1...,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...","""sir,"" said telemakhos, ""as regards your ...","("", Sir, ,, "", said, Telemakhos, ,, "", as, reg...","[sir,, said, telemakhos,, as, regards, your, q...",1
5,urn:cts:greekLit:tlg0012.tlg002.perseus-eng4:1...,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...","athena answered, ""do not try to keep me, for i...","(Athena, answered, ,, "", Do, not, try, to, kee...","[athena, answered,, do, not, try, to, keep, me...",0
...,...,...,...,...,...,...
261,urn:cts:greekLit:tlg0012.tlg002.perseus-eng4:2...,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...","laertes answered, ""would, by ...","(Laertes, answered, ,, "", Would, ,, by, \n ...","[laertes, answered,, would,, by, father, zeus,...",0
262,urn:cts:greekLit:tlg0012.tlg002.perseus-eng4:2...,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...",while they were thus busy getting their dinner...,"(While, they, were, thus, busy, getting, their...","[while, they, were, thus, busy, getting, their...",2
263,urn:cts:greekLit:tlg0012.tlg002.perseus-eng4:2...,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...","this was what he said, and more than half ...","(This, was, what, he, said, ,, and, more, than...","[this, was, what, he, said,, and, more, than, ...",1
264,urn:cts:greekLit:tlg0012.tlg002.perseus-eng4:2...,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...",now when laertes and the others had done dinne...,"(Now, when, Laertes, and, the, others, had, do...","[now, when, laertes, and, the, others, had, do...",1
